In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ipywidgets as widgets
import sys
sys.path.append("/Users/ash/utilities")
import utilities as ut

In [12]:
rcon = 7.051282051282051
width = 0.2564102564102564
v = 0.6
phi = 1.2
f = 800.0
al = 1.0
indir = "./out"
fname = "{}/v{}phi{}f{}al{}.pickle".format(indir,v,phi,f,al)
with open(fname,'rb') as handle:
    data = pickle.load(handle)
r = data['r']
phi = data['phi']
def animate(t,r,phi):
    x = r[t,:,0]
    y = r[t,:,1]
    ut.circles(x,y,0.5,c=range(len(x)),cmap='gnuplot')
    # ut.circles(0,0,rcon,'k',fc='None')
    # ut.circles(0,0,rcon+width,'k',fc='None')
    plt.gca().set_aspect(1)
widgets.interact(animate, t=(0,len(r)-1,1), r=widgets.fixed(r), phi=widgets.fixed(phi))

interactive(children=(IntSlider(value=99, description='t', max=199), Output()), _dom_classes=('widget-interact…

<function __main__.animate(t, r, phi)>

In [41]:
rcon = 7.051282051282051
width = 0.2564102564102564
v = 0.6
phi = 1.2
f = 300.0
al = 0.0
indir = "./out"
fname = "{}/v{}phi{}f{}al{}".format(indir,v,phi,f,al)
def animate(t,fname):
    x,y,phi = np.loadtxt("{}_{}.dat".format(fname,t),unpack=1)
    ut.circles(x,y,0.5,'r')
    ut.circles(0,0,rcon,'k',fc='None')
    ut.circles(0,0,rcon+width,'k',fc='None')
    plt.gca().set_aspect(1)
widgets.interact(animate, t=(0,len(r)-1,1), fname=widgets.fixed(fname))

interactive(children=(IntSlider(value=99, description='t', max=199), Output()), _dom_classes=('widget-interact…

<function __main__.animate(t, fname)>